In [1]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
id=str(sys.argv[1])
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
query = ('SELECT * from customers where id = 1')

# join para cobros
# cobros=pd.read_sql('Select cobros.* ,customers.customer,internal_orders.invoice, users.name from ((cobros inner join internal_orders on internal_orders.id = cobros.order_id) inner join customers on customers.id = internal_orders.customer_id )inner join users on cobros.capturo=users.id',cnx)


#traer datos de los pedidos
pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.alias,
coins.exchange_sell, coins.coin, coins.symbol, coins.code
from ((
    internal_orders
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
     """,cnx)
cobros=pd.read_sql("""select cobro_orders.*
                     from (((
                         cobro_orders 
    inner join cobros on cobros.id=cobro_orders.cobro_id)
    inner join internal_orders on internal_orders.id = cobros.order_id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)
facturas=pd.read_sql("""select * 
                     from ((
                         factures
    inner join internal_orders on internal_orders.id = factures.order_id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)
creditos=pd.read_sql("""select * 
                     from ((
                         credit_notes    inner join internal_orders on internal_orders.id = credit_notes.order_id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)


#Agregar columna con saldo en los cobros
pedidos=pedidos.assign(saldo=0.0)
for i in range(len(pedidos)):
    pedidos['saldo'].values[i]=cobros.loc[cobros['order_id']==pedidos['id'].values[i],'amount'].sum()
    if(cobros.loc[cobros['order_id']==pedidos['id'].values[i],'amount'].sum()<=1):
        print('here')

pedidosc=pedidos.loc[pedidos['total']-pedidos['saldo']<=1]
pedidos_interes=pedidosc.loc[pedidosc['total']<pedidosc['saldo']]
saldo_excedido=(pedidos_interes['saldo'].sum()-pedidos_interes['total'].sum())/1.16


C:\Users\hp\AppData\Local\Temp\ipykernel_5276\1799481814.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.alias,
C:\Users\hp\AppData\Local\Temp\ipykernel_5276\1799481814.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cobros=pd.read_sql("""select cobro_orders.*
C:\Users\hp\AppData\Local\Temp\ipykernel_5276\1799481814.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  facturas=pd.read_sql("""select *
C:\Users\hp\AppData\Local\Temp\ipykernel

here
here
here
here
here
here
here
here
here
here


In [13]:
customers=pd.read_sql("SELECT * from customers",cnx)

C:\Users\hp\AppData\Local\Temp\ipykernel_5276\1843457667.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers=pd.read_sql("SELECT * from customers",cnx)


In [7]:
pedidos_interes=pedidos_interes.assign(exceso=lambda x: x.saldo -x.total )

In [22]:
pedidos_interes[['invoice','customer_id','saldo','total','exceso','coin']].merge(customers[['customer','customer_id','clave']],how='inner',on='customer_id').to_excel('excesos.xlsx')

In [17]:
customers=customers.rename(columns={'id':'customer_id'})

In [25]:
pedidos_interes[['invoice','customer_id','saldo','total','exceso','coin']].merge(customers[['customer','customer_id','clave']],how='inner',on='customer_id')['clave']

0            230
1              1
2              1
3            242
4            268
5              4
6             55
7             18
8            331
9            334
10            10
11           285
12           285
13           348
14            93
15           352
16            76
17            76
18           333
19           333
Name: clave, dtype: object

In [23]:
pedidos_interes.dtypes

id                                       int64
invoice                                 object
date                                    object
customer_id                              int64
seller_id                                int64
date_delivery                           object
instalation_date                        object
shipment                                object
customer_shipping_address_id             int64
coin_id                                  int64
subtotal                               float64
iva                                    float64
total                                  float64
payment_conditions                      object
pays                                    object
observations                            object
status                                  object
authorization_id                         int64
created_at                      datetime64[ns]
updated_at                      datetime64[ns]
comision                               float64
reg_date     